In [1]:
import numpy as np
import pandas as pd
import math

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import wordnet

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

import os

In [2]:
df = pd.read_csv("balanced.csv")


In [3]:
mapping={'business':0, 'entertainment':1, 'politics':2, 'sport':3, 'tech':4}
df['CategoryId']=df['Category'].map(mapping)


### Stop Words

In [4]:
stop_words_file=open("stopwords.txt","r",encoding="utf-8")
stop_words=stop_words_file.read()
stop_words=stop_words.split("\n")

nepali_num_file=open("numbers.txt","r",encoding="utf-8")
nepali_num=nepali_num_file.read()
nepali_num=nepali_num.split(",")

nepali_suffix_file=open("suffix.txt","r",encoding="utf-8")
nepali_suffix=nepali_suffix_file.read()
nepali_suffix=nepali_suffix.split("\n")

### PreProcess

In [5]:
from Nepali_nlp import Tokenizer
def ProcessText(text):
    text=str(text)
    
    #removing \n and \ufeff
    remove=['\n','\ufeff']
    for i in remove:
        text.replace(i,'')
    
    #read stop words
    #Remove Stop Words
    word_tokens = Tokenizer().word_tokenize(text)
    filtered_list = [w for w in word_tokens if not w in stop_words]
    
    #Remove Nepali numbers
    num_filter=[]
    for i in range(0,len(filtered_list)):
        for j in range(0,len(nepali_num)):
            if nepali_num[j] in filtered_list[i]:
                num_filter.append(filtered_list[i])
                break
    for filter in num_filter:
        filtered_list.remove(filter)
    
    #Remove English numbers
    num=['0','1','2','3','4','5','6','7','8','9']
    num_filter=[]
    for i in range(0,len(filtered_list)):
        for j in range(0,len(num)):
            if num[j] in filtered_list[i]:
                num_filter.append(filtered_list[i])
                break
    for filter in num_filter:
        filtered_list.remove(filter)       
    
    #Stemming Manual
    
    suffix_filter=filtered_list
    '''
    for i in range(len(nepali_suffix)):
        for j in range(len(suffix_filter)):
            if nepali_suffix[i] in suffix_filter[j]:
                suffix_removed=suffix_filter[j].split(nepali_suffix[i])[0]
                suffix_filter.remove(suffix_filter[j])
                suffix_filter.insert(j,suffix_removed)
    '''
    stemmed_string=' '.join(suffix_filter)
    
    #stemmed_string=' '.join(filtered_list)
    
    return suffix_filter

c:\Users\Ghost\anaconda3\envs\news\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Apply Process Text in our column

In [6]:
df['News']=df['News'].apply(ProcessText)

In [8]:
from gensim.models import Word2Vec
w2v_model = Word2Vec.load("../models/sabda_to_vec_model_md")
test = w2v_model.wv.most_similar('कार्यालय')
#vector = model_W2V.wv['प्ल्याबापााकाााा']

In [106]:
len(w2v_model.wv.index_to_key)

933192

In [107]:
X_train, X_test, y_train,y_test = train_test_split(df['News'],df['Category'],test_size=0.2)


### Word2vec Model

In [108]:
words = set(w2v_model.wv.index_to_key)
X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_train],dtype=object)
X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_test],dtype=object)

In [109]:
for i,v in enumerate(X_train_vect):
    print(len(X_train.iloc[i]),len(v))
    if i>10:
        break

655 533
734 658
842 704
267 219
564 480
353 275
445 380
281 258
655 571
236 191
246 203
165 133


In [110]:

X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(100, dtype=float))
        
X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(100, dtype=float))

In [111]:
for i,v in enumerate(X_train_vect_avg):
    print(len(X_train.iloc[i]),len(v))
    if i > 10:
        break

655 100
734 100
842 100
267 100
564 100
353 100
445 100
281 100
655 100
236 100
246 100
165 100


### Fit RandomForestClassifier On top Of Word Vectors

In [112]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf_model = rf.fit(X_train_vect_avg, y_train.values.ravel())

In [113]:
y_pred = rf_model.predict(X_test_vect_avg)


In [114]:
from sklearn.metrics import precision_score, recall_score
precision = precision_score(y_test, y_pred,average="macro")
recall = recall_score(y_test, y_pred,average="macro")
print('Precision: {} / Recall: {} / Accuracy: {}'.format(round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3)))

Precision: 0.944 / Recall: 0.945 / Accuracy: 0.944


In [115]:
text = "काठमाडौँ — कांग्रेसबाट राष्ट्रपतिका उम्मेदवार रामचन्द्र पौडेलले राष्ट्रिय स्वतन्त्र पार्टी(रास्वपा)सँग समर्थन मागेका छन् । शनिबार दिउँसो रास्वपा सभापति रवि लामिछानेलाई भेटेर उनले आफूलाई समर्थन गर्न माग गरेका हुन् । भेटमा आफू लोकतन्त्रप्रति प्रतिबद्ध रहेको र लामो राजनीतिक यात्रा रहेकाले समर्थन गर्न आग्रह गरेको सभापति लामिछानेले कान्तिपुरलाई बताए । कान्तिपुरसँग संक्षिप्त कुराकानी गर्दै पौडेलसँगको भेटबारे लामिछानेले भने, 'उहाँले राष्ट्रपतिमा समर्थन गर्न आग्रह गर्नुभयो । उहाँले लोकतन्त्रप्रति प्रतिबद्ध छु। लामो राजनीतिक यात्रा छ, तपाईंहरूले समर्थन गर्नु र नगर्नुले ठूलो अर्थ राख्छ भन्नुभयो ।'त्यस्तै लामिछानेले पार्टीले निर्णय गर्नै बाँकी भएको बताएको जानकारी दिए । उनले भने, 'मैले उहाँलाई पार्टीले निर्णय गर्न बाँकी छ पार्टीगत निर्णय गर्छौं भनेँ । तपाईंलाई शुभकामना छ भनेँ ।' रास्वपाले राष्ट्रपति निर्वाचनबारे निर्णय गर्न सोमबार केन्द्रीय समिति बैठक बोलाएको छ । "

text = ProcessText(text)
text_vect = np.array([np.array([w2v_model.wv[i] for i in text if i in words])])

text_vect_avg = []
for v in text_vect:
    if v.size:
        text_vect_avg.append(v.mean(axis=0))
    else:
        text_vect_avg.append(np.zeros(300, dtype=float))
rf_model.predict(text_vect_avg)


array(['politics'], dtype=object)

In [116]:
text = "काठमाडौं । पछिल्लो समय कम्प्युटर तथा स्मार्टफोनको प्रयोग अत्यधिक बढेको छ । जसले गर्दा मानिसहरूले आफ्नो अधिक समय कम्प्युटर तथा स्मार्टफोनको स्क्रिन हेरेर बिताइरहेका  हुन्छन् । एक रिपोर्ट अनुसार विभिन्न देशमा मानिसले दिनमा औसत छ घण्टादेखि १० घण्टासम्मको समय स्क्रिनमा बताउने गरेका छन् ।​ कम्प्यारिटेकका अनुसार अमेरिकामा मात्र दैनिक एक व्यक्तिले औसतमा सात घण्टा चार मिनेट स्क्रिनमा हेरेर बिताउने गरेका छन् । यसरी अत्यधिक समय स्क्रिनमा हेरेर बिताउँदा यसले मानिसको स्वास्थ्यमा प्रतिकूल असर पारेको विभिन्न अध्ययनले देखाएका छन् । आज हामी रिसर्च गेट, हावार्ड युनिभर्सिटी जस्ता उत्कृष्ट संस्थाले तयार पारेको रिपोर्टका आधारमा धेरै समय कम्प्युटर वा मोबाइल स्क्रिनमा बिताउँदा स्वास्थ्यमा कस्ता असर पर्छ भन्ने विषयमा जानकारी दिँदैछौं । कम्प्युटर तथा मोबाइल स्क्रिनमा अधिक समय बिताउँदा किन स्वास्थ्यमा विभिन्न समस्या आउँछन् ?​ स्क्रिनमा अधिक समय बिताउँदा त्यसले आँखालाई कसरी असर गर्छ भन्ने धेरैको जिज्ञासा हुनसक्छ । तल उल्लेख गरिएका कारणले गर्दा स्क्रिनले आँखामा असर गर्छ ।"

text = ProcessText(text)
text_vect = np.array([np.array([w2v_model.wv[i] for i in text if i in words])])

text_vect_avg = []
for v in text_vect:
    if v.size:
        text_vect_avg.append(v.mean(axis=0))
    else:
        text_vect_avg.append(np.zeros(300, dtype=float))
rf_model.predict(text_vect_avg)


array(['tech'], dtype=object)